In [243]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%pip install nltk
import nltk
%pip install langdetect
from langdetect import detect

In [325]:
lyrics = pd.read_csv("songs_lyrics.csv", sep="|", na_values={"": None}, keep_default_na=False)
lyrics

,title,artist,lyrics
0,Flowers,Miley Cyrus,88 ContributorsTranslationsEspañolPortuguêsKis...
1,Cupid - Twin Ver.,FIFTY FIFTY,1 ContributorTranslationsEnglishBahasa MelayuF...
2,BESO,ROSALÍA,29 ContributorsTranslationsPortuguêsEnglishDeu...
3,Boy's a liar Pt. 2,PinkPantheress,92 ContributorsTranslationsPortuguêsTürkçeDeut...
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2 ContributorsNew Music Friday 12/02/22 Lyrics...
...,...,...,...
2380,Southbound,Beach Blvd,2 ContributorsMe & My Skate Lyrics[Skit]\nOh s...
2381,Dance with Somebody - Radio Version,Mando Diao,1 ContributorDance With Somebody - Radio Versi...
2382,Flow,Desire Machines,1 ContributorWhat Matters? Lyrics[Hook]\nLeave...
2383,Scared of the Dark,Everything Brighter,1 ContributorWay Home LyricsYa Ya Ya\n\nI hear...


In [328]:
print(f"Ci sono {(lyrics['lyrics'] == '').sum()} canzoni di cui non è stato trovato il testo.")
lyrics[lyrics["lyrics"] == ''].head()

Ci sono 54 canzoni di cui non è stato trovato il testo.


,title,artist,lyrics
16,Until I Found You (with Em Beihold) - Em Beiho...,Stephen Sanchez,
317,"Happy - From ""Despicable Me 2""",Pharrell Williams,
339,"Moves Like Jagger - Studio Recording From ""The...",Maroon 5,
446,CAN'T STOP THE FEELING! (from DreamWorks Anima...,Justin Timberlake,
684,Valerie (feat. Amy Winehouse) - Version Revisited,Mark Ronson,


In [271]:
lyrics["lyrics"][0]

"88 ContributorsTranslationsEspañolPortuguêsKiswahiliDeutschItalianoСрпскиNederlandsSvenskaPolskiTürkçeΕλληνικάFrançaisDanskРусскийالعربيةУкраїнськаBahasa IndonesiaFlowers Lyrics[Verse 1]\nWe were good, we were gold\nKinda dream that can't be sold\nWe were right 'til we weren't\nBuilt a home and watched it burn\n\n[Pre-Chorus]\nMm, I didn't wanna leave you, I didn't wanna lie\nStarted to cry, but then remembered I\n\n[Chorus]\nI can buy myself flowers\nWrite my name in the sand\nTalk to myself for hours\nSay things you don't understand\nI can take myself dancing\nAnd I can hold my own hand\nYeah, I can love me better than you can\n\n[Post-Chorus]\nCan love me better, I can love me better, baby\nCan love me better, I can love me better, baby\n\n[Verse 2]\nPaint my nails cherry-red\nMatch the roses that you left\nNo remorse, no regret\nI forgive every word you said\nYou might also like[Pre-Chorus]\nOoh, I didn't wanna leave you, baby, I didn't wanna fight\nStarted to cry, but then rememb

In [273]:
def clear_lyrics(text):
    if text != '':
        contributors, other = text.split("Contributor")
        text = other.split("Lyrics")[1].split("Embed")[0] if int(contributors) > 20 else ''
    return text

In [275]:
lyrics["lyrics"] = lyrics["lyrics"].map(clear_lyrics)
print(f"Ci sono {(lyrics['lyrics'] == '').sum()} che ora non hanno il testo.")
lyrics

,title,artist,lyrics
0,Flowers,Miley Cyrus,"[Verse 1]\nWe were good, we were gold\nKinda d..."
1,Cupid - Twin Ver.,FIFTY FIFTY,
2,BESO,ROSALÍA,"[Letra de ""BESO""]\n\n[Coro: ROSALÍA]\nYa yo ne..."
3,Boy's a liar Pt. 2,PinkPantheress,[Verse 1: PinkPantheress]\nTake a look inside ...
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,
...,...,...,...
2380,Southbound,Beach Blvd,
2381,Dance with Somebody - Radio Version,Mando Diao,
2382,Flow,Desire Machines,
2383,Scared of the Dark,Everything Brighter,


In [278]:
def detect_language(text):
    return detect(text) if text != '' else None

In [279]:
lyrics["language"] = lyrics["lyrics"].apply(detect_language)
lyrics

In [323]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(min_df=20)
mat = cvect.fit_transform(lyrics["lyrics"])
m = mat.toarray()
mt = m.sum(axis=1)
mt.argsort()

array([1192, 1522, 1519, ..., 1640, 2071, 2180], dtype=int64)